# Playing with gemmini

In [ ]:
# from nbdev.export import notebook2script; notebook2script()

In [ ]:
import gemini

In [ ]:
import pandas.api
import pandas as pd

In [ ]:
from gemini import data, engine, helpers

In [ ]:
df = data.get_ltf_candles("USDC_BTC", "30-MIN", "2020-01-02 00:00:00", "2020-11-13 00:00:00")

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
backtest = engine.backtest(df)

# Backtesting

In [ ]:
btc_usd = df.copy()
btc_usd = btc_usd.set_index('date')
btc_usd.columns = [f"{c[0].upper()}{c[1:]}" for c in btc_usd.columns]
btc_usd = btc_usd[GOOG.columns]

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy): 
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        price = self.data.Close[-1]
        _return = (price / self.ma2) - 1
        if _return < -0.02:
            abs_return = abs(_return)
            self.buy(sl=price * (1 - abs_return * 0.8), tp=price*(1 + abs_return * 0.8))


bt = Backtest(btc_usd, SmaCross, cash=10**6, commission=.002,
              exclusive_orders=False, trade_on_close=True)
stats = bt.run()

stats

In [ ]:
bt.plot()

# Zipline

In [ ]:
%load_ext zipline

In [ ]:
from zipline.finance.ledger import Ledger

In [ ]:
%%zipline --start 2017-1-1 --end 2018-1-1 --benchmark-symbol KO -o tmp.pkl

from zipline.api import order_target, record, symbol
import matplotlib.pyplot as plt

def initialize(context):
    context.i = 0
    context.asset = symbol('KO')


def handle_data(context, data):
    # Skip first 300 days to get full windows
    context.i += 1
    if context.i < 10:
        return

    # Compute averages
    # data.history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = data.history(context.asset, 'price', bar_count=5, frequency="1d").mean()
    long_mavg = data.history(context.asset, 'price', bar_count=10, frequency="1d").mean()

    # Trading logic
    if short_mavg > long_mavg:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target(context.asset, 100)
    elif short_mavg < long_mavg:
        order_target(context.asset, 0)

    # Save values for later inspection
    record(KO=data.current(context.asset, 'price'),
           short_mavg=short_mavg,
           long_mavg=long_mavg)


def analyze(context, perf):
    fig = plt.figure(figsize=[16, 8])
    ax1 = fig.add_subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax1.set_ylabel('portfolio value in $')

    ax2 = fig.add_subplot(212)
    perf['KO'].plot(ax=ax2)
    perf[['short_mavg', 'long_mavg']].plot(ax=ax2)

    perf_trans = perf.ix[[t != [] for t in perf.transactions]]
    buys = perf_trans.ix[[t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.ix[
        [t[0]['amount'] < 0 for t in perf_trans.transactions]]
    ax2.plot(buys.index, perf.short_mavg.ix[buys.index],
             '^', markersize=10, color='m')
    ax2.plot(sells.index, perf.short_mavg.ix[sells.index],
             'v', markersize=10, color='k')
    ax2.set_ylabel('price in $')
    plt.legend(loc=0)
    plt.show()

In [ ]:
perf = pd.read_pickle('tmp.pkl')

In [ ]:
perf

In [ ]:
perf.columns

In [ ]:
perf[['algorithm_period_return' ,'benchmark_period_return']]

In [ ]:
perf[['algorithm_period_return' ,'benchmark_period_return']].plot()

In [ ]:
perf[['algorithm_period_return' ,'benchmark_period_return']].iloc[500:, :]

In [ ]:
perf['max_drawdown'].plot()

In [ ]:
perf['portfolio_value'].plot()